# Import libraries and datasets

In [2]:
import pandas as pd 
import numpy as np 

In [3]:
olist_customers = pd.read_csv("/Users/phuongnguyen/Documents/Code Github/mlops-ml-ecommerce-customers/sample_files/olist_dataset/olist_customers_dataset.csv")
olist_orders = pd.read_csv("/Users/phuongnguyen/Documents/Code Github/mlops-ml-ecommerce-customers/sample_files/olist_dataset/olist_orders_dataset.csv")
olist_orders_payment = pd.read_csv("/Users/phuongnguyen/Documents/Code Github/mlops-ml-ecommerce-customers/sample_files/olist_dataset/olist_order_payments_dataset.csv")
customer_segmentation_ltv = pd.read_csv("/Users/phuongnguyen/Documents/Code Github/mlops-ml-ecommerce-customers/sample_files/customer_segmentation_ltv.csv")

In [4]:
#merge dataset orders and customers to solve unique_customer_id
orders_bycus = pd.merge(olist_orders, olist_customers, on = 'customer_id')
#merge dataset orders by customer and their payment for further use
orders_payment_bycus = pd.merge(orders_bycus, olist_orders_payment, on = 'order_id')
print(orders_payment_bycus.columns)

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'customer_unique_id', 'customer_zip_code_prefix', 'customer_city',
       'customer_state', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value'],
      dtype='object')


In [5]:
#Calculate the revenue of last month of every customer
#Filter olist_orders with purchase time in last month -> sum revenue and groupby customer_unique_id
max_date = olist_orders['order_purchase_timestamp'].max()
max_date_timestamp = pd.to_datetime(max_date)
print(max_date)

#Merge with customer_segementation_ltv customer_unique_id

2018-10-17 17:30:18


In [22]:
print(olist_orders.dtypes)

order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp                 object
order_approved_at                        object
order_delivered_carrier_date             object
order_delivered_customer_date            object
order_estimated_delivery_date            object
order_purchase_datetime          datetime64[ns]
dtype: object


In [17]:
olist_orders['order_purchase_datetime'] = pd.to_datetime(olist_orders['order_purchase_timestamp'])

In [23]:
olist_orders_3m = olist_orders[olist_orders['order_purchase_datetime'] >= '2018-7-17']

In [24]:
olist_orders.describe()

,order_purchase_datetime
count,99441
mean,2017-12-31 08:43:12.776581120
min,2016-09-04 21:15:19
25%,2017-09-12 14:46:19
50%,2018-01-18 23:04:36
75%,2018-05-04 15:42:16
max,2018-10-17 17:30:18


In [26]:
olist_orders_3m.count()

order_id                         10221
customer_id                      10221
order_status                     10221
order_purchase_timestamp         10221
order_approved_at                10148
order_delivered_carrier_date     10043
order_delivered_customer_date     9963
order_estimated_delivery_date    10221
order_purchase_datetime          10221
dtype: int64

In [27]:
orders_3m_bycus = pd.merge(olist_orders_3m, olist_customers, on = 'customer_id')

In [31]:
orders_3m_bycus.count()

order_id                         10221
customer_id                      10221
order_status                     10221
order_purchase_timestamp         10221
order_approved_at                10148
order_delivered_carrier_date     10043
order_delivered_customer_date     9963
order_estimated_delivery_date    10221
order_purchase_datetime          10221
customer_unique_id               10221
customer_zip_code_prefix         10221
customer_city                    10221
customer_state                   10221
dtype: int64

In [29]:
orders_payment_3m_bycus = pd.merge(orders_3m_bycus, olist_orders_payment, on = 'order_id')

In [30]:
orders_payment_3m_bycus.count()

order_id                         10536
customer_id                      10536
order_status                     10536
order_purchase_timestamp         10536
order_approved_at                10463
order_delivered_carrier_date     10356
order_delivered_customer_date    10273
order_estimated_delivery_date    10536
order_purchase_datetime          10536
customer_unique_id               10536
customer_zip_code_prefix         10536
customer_city                    10536
customer_state                   10536
payment_sequential               10536
payment_type                     10536
payment_installments             10536
payment_value                    10536
dtype: int64

In [33]:
revenue_3m = orders_payment_3m_bycus.groupby('customer_unique_id')['payment_value'].sum().reset_index()

In [34]:
revenue_3m.columns = ['customer_unique_id', 'revenue_3m']

In [35]:
revenue_3m.head(5)

,customer_unique_id,revenue_3m
0,000ec5bff359e1c0ad76a81a45cb598f,27.75
1,000fbf0473c10fc1ab6f8d2d286ce20c,362.38
2,0015752e079902b12cd00b9b7596276b,74.82
3,00172711b30d52eea8b313a7f2cced02,244.14
4,001928b561575b2821c92254a2327d06,164.81


In [38]:
rfm_revenue_3m = pd.merge(customer_segmentation_ltv, revenue_3m, on = 'customer_unique_id', how='left')
rfm_revenue_3m = rfm_revenue_3m.fillna(0)

In [48]:
rfm_revenue_3m.drop(['Unnamed: 0'], axis=1)

,customer_unique_id,recency,recency_cluster,frequency,frequency_cluster,revenue,revenue_cluster,rfm_score,Segment,revenue_3m,ltv_cluster
0,0000366f3b9a7992bf8c76cfdf3221e2,160,3,1,0,141.90,0,3,Low-Value,0.00,0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,163,3,1,0,27.19,0,3,Low-Value,0.00,0
2,000e309254ab1fc5ba99dd469d36bdb4,114,3,1,0,78.42,0,3,Low-Value,0.00,0
3,000ec5bff359e1c0ad76a81a45cb598f,57,3,1,0,27.75,0,3,Low-Value,27.75,0
4,000ed48ceeb6f4bf8ad021a10a3c7b43,150,3,1,0,20.75,0,3,Low-Value,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...
96090,fff1bdd5c5e37ca79dd74deeb91aa5b6,234,2,1,0,172.98,0,2,Low-Value,0.00,0
96091,fff2ae16b99c6f3c785f0e052f2a9cfb,180,2,1,0,200.90,0,2,Low-Value,0.00,0
96092,fff7219c86179ca6441b8f37823ba3d3,293,2,1,0,265.80,1,3,Low-Value,0.00,0
96093,fffbf87b7a1a6fa8b03f081c5f51a201,293,2,1,0,167.32,0,2,Low-Value,0.00,0


In [45]:
from sklearn.cluster import KMeans

In [49]:
#Creating 3 clusters
kmeans = KMeans(n_clusters = 3)
kmeans.fit(rfm_revenue_3m[['revenue_3m']])
rfm_revenue_3m['ltv_cluster'] = kmeans.predict(rfm_revenue_3m[['revenue_3m']])

In [50]:
rfm_revenue_3m.head(5)

,Unnamed: 0,customer_unique_id,recency,recency_cluster,frequency,frequency_cluster,revenue,revenue_cluster,rfm_score,Segment,revenue_3m,ltv_cluster
0,0,0000366f3b9a7992bf8c76cfdf3221e2,160,3,1,0,141.90,0,3,Low-Value,0.00,0
1,1,0000b849f77a49e4a4ce2b2a4ca5be3f,163,3,1,0,27.19,0,3,Low-Value,0.00,0
2,2,000e309254ab1fc5ba99dd469d36bdb4,114,3,1,0,78.42,0,3,Low-Value,0.00,0
3,3,000ec5bff359e1c0ad76a81a45cb598f,57,3,1,0,27.75,0,3,Low-Value,27.75,0
4,4,000ed48ceeb6f4bf8ad021a10a3c7b43,150,3,1,0,20.75,0,3,Low-Value,0.00,0


In [52]:
rfm_revenue_3m.drop(['Unnamed: 0'], axis=1)
rfm_revenue_3m.head(5)

,Unnamed: 0,customer_unique_id,recency,recency_cluster,frequency,frequency_cluster,revenue,revenue_cluster,rfm_score,Segment,revenue_3m,ltv_cluster
0,0,0000366f3b9a7992bf8c76cfdf3221e2,160,3,1,0,141.90,0,3,Low-Value,0.00,0
1,1,0000b849f77a49e4a4ce2b2a4ca5be3f,163,3,1,0,27.19,0,3,Low-Value,0.00,0
2,2,000e309254ab1fc5ba99dd469d36bdb4,114,3,1,0,78.42,0,3,Low-Value,0.00,0
3,3,000ec5bff359e1c0ad76a81a45cb598f,57,3,1,0,27.75,0,3,Low-Value,27.75,0
4,4,000ed48ceeb6f4bf8ad021a10a3c7b43,150,3,1,0,20.75,0,3,Low-Value,0.00,0


In [53]:
rfm_revenue_3m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96095 entries, 0 to 96094
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          96095 non-null  int64  
 1   customer_unique_id  96095 non-null  object 
 2   recency             96095 non-null  int64  
 3   recency_cluster     96095 non-null  int64  
 4   frequency           96095 non-null  int64  
 5   frequency_cluster   96095 non-null  int64  
 6   revenue             96095 non-null  float64
 7   revenue_cluster     96095 non-null  int64  
 8   rfm_score           96095 non-null  int64  
 9   Segment             96095 non-null  object 
 10  revenue_3m          96095 non-null  float64
 11  ltv_cluster         96095 non-null  int32  
dtypes: float64(2), int32(1), int64(7), object(2)
memory usage: 8.4+ MB


In [54]:
df_ML = rfm_revenue_3m.copy()
df_ML = df_ML.drop(['Unnamed: 0'], axis=1)

In [55]:
df_ML.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96095 entries, 0 to 96094
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   customer_unique_id  96095 non-null  object 
 1   recency             96095 non-null  int64  
 2   recency_cluster     96095 non-null  int64  
 3   frequency           96095 non-null  int64  
 4   frequency_cluster   96095 non-null  int64  
 5   revenue             96095 non-null  float64
 6   revenue_cluster     96095 non-null  int64  
 7   rfm_score           96095 non-null  int64  
 8   Segment             96095 non-null  object 
 9   revenue_3m          96095 non-null  float64
 10  ltv_cluster         96095 non-null  int32  
dtypes: float64(2), int32(1), int64(6), object(2)
memory usage: 7.7+ MB


In [56]:
#Categorical columns
for i in ["customer_unique_id","recency_cluster","frequency_cluster", "revenue_cluster", "Segment", "ltv_cluster"]:
    df_ML[i]= df_ML[i].astype('object')

In [57]:
#Numerical columns
Num= df_ML.select_dtypes(exclude='object').columns.tolist()
Num

['recency', 'frequency', 'revenue', 'rfm_score', 'revenue_3m']

In [58]:
#Apply StandardScaler for numerical columns
from sklearn.preprocessing import StandardScaler 
scalerNum = StandardScaler()

for cols in Num:
    df_ML[cols] = scalerNum.fit_transform(df_ML[[cols]])

Categorical columns:

In [59]:
Cat = df_ML.select_dtypes(include='object').columns.tolist()
Cat

['customer_unique_id',
 'recency_cluster',
 'frequency_cluster',
 'revenue_cluster',
 'Segment',
 'ltv_cluster']

In [60]:
#Apply LabelEncoder for categorical columns
from sklearn.preprocessing import LabelEncoder
df_ML = df_ML.apply(lambda x: LabelEncoder().fit_transform(x) if x.dtype == 'object' else x)
df_ML.head()

,customer_unique_id,recency,recency_cluster,frequency,frequency_cluster,revenue,revenue_cluster,rfm_score,Segment,revenue_3m,ltv_cluster
0,0,-0.832626,3,-0.162369,0,-0.106704,0,0.907579,1,-0.178923,0
1,1,-0.813071,3,-0.162369,0,-0.602366,0,0.907579,1,-0.178923,0
2,19,-1.132482,3,-0.162369,0,-0.381001,0,0.907579,1,-0.178923,0
3,20,-1.504043,3,-0.162369,0,-0.599946,0,0.907579,1,0.111476,0
4,21,-0.897812,3,-0.162369,0,-0.630193,0,0.907579,1,-0.178923,0


# Data preparation

In [61]:
#Create X, y with X for feature set and y for lable set - CLV
X = df_ML.drop(['ltv_cluster', 'revenue_3m'], axis = 1)
y = df_ML['ltv_cluster']

In [71]:
#Train Test Split
from sklearn.model_selection import KFold, cross_val_score, train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state =0 )

In [72]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier 
import xgboost as xgb

# Model training

In [73]:
models = {
    'K-Nearest Neighbors' : KNeighborsClassifier(),
    'Logistic Regression' : LogisticRegression(),
    'Random Forest' : RandomForestClassifier(),
    'Gradient Boosting' : GradientBoostingClassifier(),
    'XG Boosting' : xgb.XGBClassifier()
}

In [74]:
%%time
for name, model in models.items():
    print(f'{name} training started')
    model.fit(X_train, y_train)
    print(f'{name} trained')
    print("*"*40)
print("FINISHED")

K-Nearest Neighbors training started
K-Nearest Neighbors trained
****************************************
Logistic Regression training started


/Users/phuongnguyen/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression trained
****************************************
Random Forest training started
Random Forest trained
****************************************
Gradient Boosting training started
Gradient Boosting trained
****************************************
XG Boosting training started
XG Boosting trained
****************************************
FINISHED
CPU times: user 31.4 s, sys: 1.77 s, total: 33.2 s
Wall time: 23.9 s


# Evaluation

In [75]:
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix as CM
from sklearn.metrics import classification_report

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_classification

In [76]:
for name in models:
    acc_score= round(accuracy_score(y_test, models.get(name).predict(X_test)), 3)
    print(f'{name} accuracy score :  {acc_score}')


K-Nearest Neighbors accuracy score :  0.959
Logistic Regression accuracy score :  0.958
Random Forest accuracy score :  0.998
Gradient Boosting accuracy score :  0.997
XG Boosting accuracy score :  0.998


In [77]:
for name in models:
    print(f'{name} classification report')
    print("-------------------------------")
    print(classification_report(y_test, models.get(name).predict(X_test)))
    print("******************************")
    print(" ")

K-Nearest Neighbors classification report
-------------------------------


/Users/phuongnguyen/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/phuongnguyen/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/phuongnguyen/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitaliz

              precision    recall  f1-score   support

           0       0.96      1.00      0.98     30429
           1       0.00      0.00      0.00        41
           2       0.14      0.00      0.00      1242

    accuracy                           0.96     31712
   macro avg       0.37      0.33      0.33     31712
weighted avg       0.93      0.96      0.94     31712

******************************
 
Logistic Regression classification report
-------------------------------
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     30429
           1       0.20      0.07      0.11        41
           2       0.30      0.05      0.08      1242

    accuracy                           0.96     31712
   macro avg       0.49      0.37      0.39     31712
weighted avg       0.94      0.96      0.94     31712

******************************
 
Random Forest classification report
-------------------------------
              precision    rec

Reference:
- Kaggle notebook: https://www.kaggle.com/code/yaseminturker/customer-lifetime-value-clv-prediction